In [1]:
%matplotlib inline
import json
import matplotlib.pyplot as plt
import pandas as pd
from pandas.io.json import json_normalize
import time
import numpy as np
import seaborn as sns
import re
import os.path

In [3]:
path = "executions-bkp-gcloudeu-28-11.jsonl"
raw_data = pd.read_json(path, lines=True)
raw_data.head()

,_id,payload,timestamp,topic
0,{'$oid': '5d274d843299fa6236b8bc28'},{'responseStamp': 1},2019-07-11 14:53:56.866,data/case/332f02b9-093e-4a2f-8d93-28022c275e8b...
1,{'$oid': '5d274d853299fa6236b8bc29'},"{'name': 'Zombie Venom 09', 'source': '# 1 (ca...",2019-07-11 14:53:57.587,data/case/332f02b9-093e-4a2f-8d93-28022c275e8b...
2,{'$oid': '5d274e633299fa6236b8bc2a'},{'responseStamp': 1},2019-07-11 14:57:39.979,data/case/00f14675-854c-4c58-95eb-8dd414f525d3...
3,{'$oid': '5d274e643299fa6236b8bc2b'},"{'name': 'Zombie Venom 08', 'source': '# 1 (ca...",2019-07-11 14:57:40.069,data/case/00f14675-854c-4c58-95eb-8dd414f525d3...
4,{'$oid': '5d274e643299fa6236b8bc2c'},{'responseStamp': 2},2019-07-11 14:57:40.085,data/theme/zombie.note/get


In [4]:
nl_experiment = raw_data.loc[lambda df: df.timestamp > '2019-11-28 7:30:00']

In [5]:
if os.path.exists('nl_experiment1.json'):
    print("Experiment data file already exists")
else:
    nl_experiment.to_json('nl_experiment1.json', orient='records')

In [6]:
def reconstruct_users_track(log):
    users_tracks = dict()
    for index, row in log.iterrows():
        payload = row.payload
        try:
            user = payload['userid']
            if user in users_tracks:
                users_tracks[user].append(row)
            else:
                users_tracks.update({user: [row]})
        except:
            pass
        
        try:
            user = payload['track']['userid'] 
            if user in users_tracks:
                users_tracks[user].append(row)
            else:
                users_tracks.update({user: [row]})
        except:
            pass

    return users_tracks

In [7]:
nl_experiment.head(10)

,_id,payload,timestamp,topic
22113,{'$oid': '5ddf8655cd683579f92534ee'},"{'email': 'A33@ds4h.org', 'password': 'a33', '...",2019-11-28 08:33:25.387,data/user/login
22114,{'$oid': '5ddf8655cd683579f92534ef'},"{'userid': 93, 'token': 'eyJhbGciOiJIUzI1NiIsI...",2019-11-28 08:33:25.551,data/user/login/response/2
22115,{'$oid': '5ddf8655cd683579f92534f0'},"{'filterBy': 'user', 'filter': 93, 'responseSt...",2019-11-28 08:33:25.614,data/case/*/list
22116,{'$oid': '5ddf8655cd683579f92534f1'},[{'id': 'a46c78ec-9973-4792-a4ec-d9a9ef0e4978'...,2019-11-28 08:33:25.729,data/case/*/list/response/3
22117,{'$oid': '5ddf8655cd683579f92534f2'},{'responseStamp': 4},2019-11-28 08:33:25.754,data/case/a46c78ec-9973-4792-a4ec-d9a9ef0e4978...
22118,{'$oid': '5ddf8655cd683579f92534f3'},"{'name': 'EKG-First', 'source': 'EKG-Descripti...",2019-11-28 08:33:25.883,data/case/a46c78ec-9973-4792-a4ec-d9a9ef0e4978...
22119,{'$oid': '5ddf8656cd683579f92534f4'},{},2019-11-28 08:33:26.004,knot/<</navigate
22120,{'$oid': '5ddf8656cd683579f92534f5'},"{'parameter': 'EKG first', 'track': {'userid':...",2019-11-28 08:33:26.033,2019-11-28T08:33:25.342Z-980cae4f-15d1-9d08-11...
22121,{'$oid': '5ddf8656cd683579f92534f6'},"{'track': {'userid': 93, 'caseid': 'a46c78ec-9...",2019-11-28 08:33:26.036,2019-11-28T08:33:25.342Z-980cae4f-15d1-9d08-11...
22122,{'$oid': '5ddf8656cd683579f92534f7'},"{'track': {'userid': 93, 'caseid': 'a46c78ec-9...",2019-11-28 08:33:26.082,2019-11-28T08:33:25.342Z-980cae4f-15d1-9d08-11...


In [8]:
nl_experiment

,_id,payload,timestamp,topic
22113,{'$oid': '5ddf8655cd683579f92534ee'},"{'email': 'A33@ds4h.org', 'password': 'a33', '...",2019-11-28 08:33:25.387,data/user/login
22114,{'$oid': '5ddf8655cd683579f92534ef'},"{'userid': 93, 'token': 'eyJhbGciOiJIUzI1NiIsI...",2019-11-28 08:33:25.551,data/user/login/response/2
22115,{'$oid': '5ddf8655cd683579f92534f0'},"{'filterBy': 'user', 'filter': 93, 'responseSt...",2019-11-28 08:33:25.614,data/case/*/list
22116,{'$oid': '5ddf8655cd683579f92534f1'},[{'id': 'a46c78ec-9973-4792-a4ec-d9a9ef0e4978'...,2019-11-28 08:33:25.729,data/case/*/list/response/3
22117,{'$oid': '5ddf8655cd683579f92534f2'},{'responseStamp': 4},2019-11-28 08:33:25.754,data/case/a46c78ec-9973-4792-a4ec-d9a9ef0e4978...
22118,{'$oid': '5ddf8655cd683579f92534f3'},"{'name': 'EKG-First', 'source': 'EKG-Descripti...",2019-11-28 08:33:25.883,data/case/a46c78ec-9973-4792-a4ec-d9a9ef0e4978...
22119,{'$oid': '5ddf8656cd683579f92534f4'},{},2019-11-28 08:33:26.004,knot/<</navigate
22120,{'$oid': '5ddf8656cd683579f92534f5'},"{'parameter': 'EKG first', 'track': {'userid':...",2019-11-28 08:33:26.033,2019-11-28T08:33:25.342Z-980cae4f-15d1-9d08-11...
22121,{'$oid': '5ddf8656cd683579f92534f6'},"{'track': {'userid': 93, 'caseid': 'a46c78ec-9...",2019-11-28 08:33:26.036,2019-11-28T08:33:25.342Z-980cae4f-15d1-9d08-11...
22122,{'$oid': '5ddf8656cd683579f92534f7'},"{'track': {'userid': 93, 'caseid': 'a46c78ec-9...",2019-11-28 08:33:26.082,2019-11-28T08:33:25.342Z-980cae4f-15d1-9d08-11...


In [9]:
users_track = reconstruct_users_track(nl_experiment)

In [11]:
print(sorted(users_track.keys()))

[1, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 115, 116]


In [12]:
def split_groups(users_track):
    group_A = dict()
    group_B = dict()
    
    for user in users_track.keys():
        for log in users_track[user]:
            try:
                if log['payload']['parameter'] == "Description first":
                    group_A[user] = users_track[user]
                if log['payload']['parameter'] == "EKG first":
                    group_B[user] = users_track[user]
            except:
                pass
    return group_A, group_B

In [13]:
group_A, group_B = split_groups(users_track)
print("Users Group A - Description First")
print(sorted(list(group_A.keys())))
print("Users Group B - EKG First")
print(sorted(list(group_B.keys())))

Users Group A - Description First
[1, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 87]
Users Group B - EKG First
[86, 87, 88, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 115, 116]


In [14]:
# Manually adding user 89 whose don't have Group parameter
# group_B[89]=users_track[89]
#Deleting admin user and user in both groups
del(group_A[1])
del(group_A[87])
del(group_B[87])

In [15]:
print(len(list(group_A.keys())))
print(len(list(group_B.keys())))

24
28


In [16]:
def users_answers(users_track):
    
    users_answers = dict()
    
    for user in users_track.keys():
        users_answers[user] = pd.DataFrame(columns=["Variable", "Answer", "CaseId"])

        for log in users_track[user]:
            if re.search('/var/[\w+.]+/set', log['topic']):
                var_topic = re.search('/var/[\w+.]+/set', log['topic']).group()
                answer = log['payload']['content']
                case_id = log['payload']['track']['caseid']
                elapsed_time = 0
                row = {"Variable": var_topic, "Answer": answer, "CaseId": case_id}
                users_answers[user] = users_answers[user].append(row, ignore_index=True)
                
    return users_answers

In [17]:
answers_groupA = users_answers(group_A)
answers_groupB = users_answers(group_B)

In [18]:
def clean_variables_track(answers_track, columns):
    
    users_answers_clean = dict()
    
    for user in answers_track.keys():
        cleaned_track = pd.DataFrame(columns=["Variable", "Answer", "CaseId"])
        group_track = answers_track[user].groupby(columns)
        
        for name, group in group_track:
            cleaned_track = cleaned_track.append(group_track.get_group(name).iloc[-1])
#             print(cleaned_track)
            
        users_answers_clean[user] = cleaned_track.sort_values(by=["CaseId","Variable"], ascending=False)
    
    return users_answers_clean

In [19]:
groupA_clean = clean_variables_track(answers_groupA, columns=["Variable", "CaseId"])
groupB_clean = clean_variables_track(answers_groupB, columns=["Variable", "CaseId"])

In [20]:
cases_dict = {"Case-01":"5c0cf92e-ebe3-4f83-bb3b-11846d0f3da8",
              "Case-02":"71cb08b0-aa66-402f-8d35-027e01bd541f",
              "Case-03":"1edff734-1090-4d44-bd52-963156b6b41d",
              "Case-07":"9c5b0ab8-33c8-4bcf-bcb4-d888327b870b",
              "Case-08":"28688ea2-f8d8-4664-80f6-0926720990fb",
              "Case-09":"ec4be4fd-4e48-4e84-ba8e-36c53b415698",
              "Case-12":"280fa4a7-1e79-4097-a961-7fda24d7d9bf",
              "Case-13":"7fbf35be-79fa-4087-94cd-ccc8030ac305",
              "Case-15":"a775bf21-b15c-44d1-a120-61611bc7960c",
              "Case-16":"b90056f9-e5e2-4281-bcfe-75ab62033b2b",
              "Case-17":"070e904b-fe21-4629-83b2-4f9ea49bccce",
              "Case-19":"64634e19-ea0a-4060-9656-1252b5d6acb5"
             }
cases_name = pd.DataFrame(cases_dict.items(), columns=["Case", "CaseId"])

In [21]:
topic_dict = {"EKG Hypothesis":"/var/EKG_Cycle.Generate_hypothesis.hypothesis/set",
              "EKG Confidence":"/var/EKG_Cycle.Generate_hypothesis.confidence/set",
              "Description Hypothesis":"/var/Description_Cycle.Generate_hypothesis.hypothesis/set",
              "Description Confidence":"/var/Description_Cycle.Generate_hypothesis.confidence/set"
             }
topic_names = pd.DataFrame(topic_dict.items(), columns=["Question", "Variable"])

In [22]:
def beautify_case_topic_name(users_track, topic_names, cases_name):
    
    for user in users_track.keys():
        df_merged = pd.merge(users_track[user],cases_name, on="CaseId", how="outer")
        df_merged = pd.merge(df_merged,topic_names, on="Variable", how="outer")
        df_merged.drop(columns=["Variable", "CaseId"], inplace=True)
        df_merged = df_merged[["Question","Answer","Case"]]
        df_merged = df_merged.sort_values(by=["Case","Question"], ascending=False)
        df_merged["Answer"] = df_merged["Answer"].str.lower()
        df_merged["Answer"] = df_merged["Answer"].str.strip()
        users_track[user] = df_merged
    
    return users_track

In [23]:
groupA_beauti = beautify_case_topic_name(groupA_clean, topic_names, cases_name)

In [24]:
groupB_beauti = beautify_case_topic_name(groupB_clean, topic_names, cases_name)

In [25]:
groupA_beauti[61]

,Question,Answer,Case
6,EKG Hypothesis,acs,Case-19
18,EKG Confidence,80,Case-19
30,Description Hypothesis,acs,Case-19
42,Description Confidence,72,Case-19
11,EKG Hypothesis,acs,Case-17
23,EKG Confidence,80,Case-17
35,Description Hypothesis,hyperventilation,Case-17
47,Description Confidence,61,Case-17
1,EKG Hypothesis,acs 3vd,Case-16
13,EKG Confidence,89,Case-16


In [26]:
def all_answers_group(users_track, start_df=None):
    
    all_answers_group = pd.DataFrame(columns=["Question","Answer","Case"])
    
    if start_df is not None:
        all_answers_group = start_df
    
    for user in users_track:
        all_answers_group = pd.concat([all_answers_group, users_track[user]])
    
    return all_answers_group

In [27]:
all_answers_groupA = all_answers_group(groupA_beauti)

In [28]:
all_answers_groupA

,Question,Answer,Case
6,EKG Hypothesis,iap,Case-19
18,EKG Confidence,79,Case-19
30,Description Hypothesis,iap,Case-19
42,Description Confidence,76,Case-19
11,EKG Hypothesis,ap,Case-17
23,EKG Confidence,18,Case-17
35,Description Hypothesis,oesofagus spasme,Case-17
47,Description Confidence,62,Case-17
1,EKG Hypothesis,ap,Case-16
13,EKG Confidence,68,Case-16


In [29]:
all_answers_groupB = all_answers_group(groupB_beauti)

In [30]:
all_answers_groupB

,Question,Answer,Case
6,EKG Hypothesis,atypisch,Case-19
18,EKG Confidence,68,Case-19
11,EKG Hypothesis,ischemie,Case-17
23,EKG Confidence,68,Case-17
34,Description Hypothesis,ap,Case-17
45,Description Confidence,59,Case-17
1,EKG Hypothesis,nstemi,Case-16
13,EKG Confidence,73,Case-16
25,Description Hypothesis,acs,Case-16
36,Description Confidence,86,Case-16


In [31]:
all_answers_experiment = all_answers_group(groupB_beauti,start_df=all_answers_groupA)

In [32]:
all_answers_experiment

,Question,Answer,Case
6,EKG Hypothesis,iap,Case-19
18,EKG Confidence,79,Case-19
30,Description Hypothesis,iap,Case-19
42,Description Confidence,76,Case-19
11,EKG Hypothesis,ap,Case-17
23,EKG Confidence,18,Case-17
35,Description Hypothesis,oesofagus spasme,Case-17
47,Description Confidence,62,Case-17
1,EKG Hypothesis,ap,Case-16
13,EKG Confidence,68,Case-16


In [33]:
hypothesis_groupA = all_answers_groupA[all_answers_groupA["Question"].str.contains("Hypothesis", na=False)]
hypothesis_groupB = all_answers_groupB[all_answers_groupB["Question"].str.contains("Hypothesis", na=False)]
all_hypothesis = all_answers_experiment[all_answers_experiment["Question"].str.contains("Hypothesis", na=False)]
confidence_groupA = all_answers_groupA[all_answers_groupA["Question"].str.contains("Confidence", na=False)]
confidence_groupB = all_answers_groupB[all_answers_groupB["Question"].str.contains("Confidence", na=False)]
all_confidence = all_answers_experiment[all_answers_experiment["Question"].str.contains("Confidence", na=False)]

In [34]:
answers_bycase_groupA_count = hypothesis_groupA.groupby("Case").apply(lambda x: x["Answer"].value_counts())

In [35]:
answers_bycase_groupA_count

Case                                                            
Case-01  acs                                                        17
         nstemi                                                     12
         nste-acs                                                    4
         reflux                                                      3
         recidief infarct                                            1
         acs-nstemi                                                  1
         reflux dd acs                                               1
         nstemi oud ecg?                                             1
         zuurbranden                                                 1
         angina pectoris                                             1
         gastro-intestinaal dd acs                                   1
         gastritis                                                   1
         oud ecg? dd blijft gelijk                                   1
         oud

In [36]:
answers_bycase_groupA_count.to_excel("hypothesis_groupDescription_count.xlsx")

In [37]:
answers_bycase_groupB_count = hypothesis_groupB.groupby("Case").apply(lambda x: x["Answer"].value_counts())
answers_bycase_groupB_count

Case                                                                                                                       
Case-01  nstemi                                                                                                                17
         acs                                                                                                                    8
         inferolateraal negatieve ts en minimale st depressie (geen oud ecg), verdenking nstemi, stenose plcx restenose rca     2
         stabiele angina pectoris                                                                                               2
         doorgemaakt inferolateraal infarct                                                                                     2
         doorgemaakt onderwandinfarct                                                                                           1
         instabiele ap                                                                          

In [38]:
answers_bycase_groupB_count.to_excel("hypothesis_groupEKG_count.xlsx")

In [39]:
answers_bycase_count = all_hypothesis.groupby("Case").apply(lambda x: x["Answer"].value_counts())
answers_bycase_count

Case                                                                                                                       
Case-01  nstemi                                                                                                                29
         acs                                                                                                                   25
         nste-acs                                                                                                               4
         reflux                                                                                                                 3
         stabiele angina pectoris                                                                                               2
         inferolateraal negatieve ts en minimale st depressie (geen oud ecg), verdenking nstemi, stenose plcx restenose rca     2
         doorgemaakt inferolateraal infarct                                                     

In [40]:
answers_bycase_count.to_excel("hypothesis_experiment_count.xlsx")

In [41]:
answers_bycase_groupA = hypothesis_groupA.groupby("Case").apply(lambda x: x["Answer"].unique())
answers_bycase_groupB = hypothesis_groupB.groupby("Case").apply(lambda x: x["Answer"].unique())
answers_bycase = all_hypothesis.groupby("Case").apply(lambda x: x["Answer"].unique())

In [42]:
answers_bycase_groupA

Case
Case-01    [acs, nste-acs, acs-nsteni, acs-nstemi, oud ec...
Case-02    [gastro esofagaal, gastro-esofagaal, maagklach...
Case-03    [tendomyalgeen, hyperventilatie, pericarditis ...
Case-07    [pericarditis, pericaridits, costomyalgeen, te...
Case-08    [ap, acs, nste-acs, iap, acs-nstemi, stemi pro...
Case-09    [pneumonie, acs, stemi, nste-acs, stemi latera...
Case-12    [hvs, costomyalgeen, acs, reflux, reflux/maagk...
Case-13    [tendomyalgeen, acs, costomyalgeen, ap, ap-kla...
Case-15    [iap, acs, nste-acs, infarct, acs-nstemi, acs ...
Case-16    [ap, acs, nste-acs panischemie, iap, acs-nstem...
Case-17    [ap, oesofagus spasme, acs, hyperventilatie, t...
Case-19    [iap, acs, nste-acs, nstemi, gastritis, psycho...
dtype: object

In [32]:
answers_bycase_groupA.to_excel("hypothesis_groupA_bycase.xlsx")

In [43]:
answers_bycase_groupB

Case
Case-01    [nstemi, doorgemaakt inferolateraal infarct, a...
Case-02    [ge reflux, inferior stemi, reflux, pericardit...
Case-03    [lvh, hyperventilatie, stemi, costo-myalgeen, ...
Case-07    [pericarditis, stemi, acs, inferoseptaal infar...
Case-08    [infarct, iap, stemi septaal, nstemi, panische...
Case-09    [longembolie, misschien stemi lateraal, doorge...
Case-12    [atypisch, geen ischemie, nstemi, niet cardiaa...
Case-13    [la dilatatie, nstemi, mitralisklepinsufficien...
Case-15    [normaal, iap, normaal ecg, mogelijk inferior ...
Case-16    [nstemi, acs, hoofdstamstenose, demand ischemi...
Case-17    [ischemie, ap, nstemi, doorgemaakt ipl infarct...
Case-19    [atypisch, doorgemaakte ischemie inferior, nst...
dtype: object

In [34]:
answers_bycase_groupB.to_excel("hypothesis_groupB_bycase.xlsx")

In [44]:
answers_bycase

Case
Case-01    [acs, nste-acs, acs-nsteni, acs-nstemi, oud ec...
Case-02    [gastro esofagaal, gastro-esofagaal, maagklach...
Case-03    [tendomyalgeen, hyperventilatie, pericarditis ...
Case-07    [pericarditis, pericaridits, costomyalgeen, te...
Case-08    [ap, acs, nste-acs, iap, acs-nstemi, stemi pro...
Case-09    [pneumonie, acs, stemi, nste-acs, stemi latera...
Case-12    [hvs, costomyalgeen, acs, reflux, reflux/maagk...
Case-13    [tendomyalgeen, acs, costomyalgeen, ap, ap-kla...
Case-15    [iap, acs, nste-acs, infarct, acs-nstemi, acs ...
Case-16    [ap, acs, nste-acs panischemie, iap, acs-nstem...
Case-17    [ap, oesofagus spasme, acs, hyperventilatie, t...
Case-19    [iap, acs, nste-acs, nstemi, gastritis, psycho...
dtype: object

In [43]:
answers_bycase.to_excel("hypothesis_experiment_bycase.xlsx")

In [45]:
all_confidence.astype({'Answer':'float64'})

,Question,Answer,Case
18,EKG Confidence,79.0,Case-19
42,Description Confidence,76.0,Case-19
23,EKG Confidence,18.0,Case-17
47,Description Confidence,62.0,Case-17
13,EKG Confidence,68.0,Case-16
37,Description Confidence,73.0,Case-16
14,EKG Confidence,59.0,Case-15
38,Description Confidence,64.0,Case-15
16,EKG Confidence,66.0,Case-13
40,Description Confidence,64.0,Case-13


In [46]:
mean_std_all = all_confidence.groupby(["Case","Question"]).apply(lambda df: (df.Answer.astype('float64').mean(), df.Answer.astype('float64').std()))

In [47]:
mean_std_all

Case     Question              
Case-01  Description Confidence                 (58.25, 19.229139448284435)
         EKG Confidence             (59.73076923076923, 18.151989449239693)
Case-02  Description Confidence     (69.66666666666667, 15.367064347710224)
         EKG Confidence              (63.17307692307692, 17.54961722821076)
Case-03  Description Confidence      (67.6470588235294, 19.399818070705475)
         EKG Confidence             (60.68627450980392, 23.559703050826787)
Case-07  Description Confidence     (72.26923076923077, 18.985149391982606)
         EKG Confidence              (69.42307692307692, 21.86890186493781)
Case-08  Description Confidence                 (72.75, 14.215256923874598)
         EKG Confidence              (68.73076923076923, 22.35113313319614)
Case-09  Description Confidence                 (58.26, 22.642573229405254)
         EKG Confidence              (55.80392156862745, 26.45147981330582)
Case-12  Description Confidence     (56.91836734693877, 

In [49]:
mean_std_all.to_excel("Mean-Std-all.xlsx")

In [50]:
mean_std_groupA = confidence_groupA.groupby(["Case","Question"]).apply(lambda df: (df.Answer.astype('float64').mean(), df.Answer.astype('float64').std()))

In [51]:
mean_std_groupA

Case     Question              
Case-01  Description Confidence      (54.541666666666664, 20.3811062691313)
         EKG Confidence                          (61.5, 17.433101946534382)
Case-02  Description Confidence                  (68.25, 16.04138669057391)
         EKG Confidence             (66.41666666666667, 15.021482201708054)
Case-03  Description Confidence     (70.78260869565217, 16.681390071080926)
         EKG Confidence              (73.26086956521739, 17.76671400350142)
Case-07  Description Confidence     (65.70833333333333, 19.116301185493302)
         EKG Confidence             (82.04166666666667, 18.355141837753745)
Case-08  Description Confidence      (74.29166666666667, 12.21360313131532)
         EKG Confidence              (83.95833333333333, 14.97382982774096)
Case-09  Description Confidence    (58.458333333333336, 21.500210649456776)
         EKG Confidence              (65.08333333333333, 27.29057277374076)
Case-12  Description Confidence     (55.30434782608695, 

In [52]:
mean_std_groupA.to_excel("Mean-Std-GroupA.xlsx")

In [53]:
mean_std_groupB = confidence_groupB.groupby(["Case","Question"]).apply(lambda df: (df.Answer.astype('float64').mean(), df.Answer.astype('float64').std()))

In [54]:
mean_std_groupB

Case     Question              
Case-01  Description Confidence     (61.42857142857143, 17.941114733583245)
         EKG Confidence            (58.214285714285715, 18.929414975713097)
Case-02  Description Confidence     (70.92592592592592, 14.932993738507427)
         EKG Confidence            (60.392857142857146, 19.291793084710413)
Case-03  Description Confidence      (65.07142857142857, 21.32973680000932)
         EKG Confidence             (50.357142857142854, 22.92049751530726)
Case-07  Description Confidence     (77.89285714285714, 17.276272072399667)
         EKG Confidence             (58.607142857142854, 18.80177155899448)
Case-08  Description Confidence     (71.42857142857143, 15.833458646120647)
         EKG Confidence             (55.67857142857143, 19.231635918606095)
Case-09  Description Confidence     (58.07692307692308, 24.073093821813725)
         EKG Confidence             (47.55555555555556, 23.163853361738038)
Case-12  Description Confidence      (58.34615384615385,

In [55]:
mean_std_groupB.to_excel("Mean-Std-GroupB.xlsx")

In [47]:
def save_as_xlsx(users_dict, path, filename_p=""):
    
    filename = path + filename_p
    
    for user in users_dict.keys():
        
        filename += str(user) + ".xlsx"
        users_dict[user].to_excel(filename, index=False)
        filename = path + filename_p

In [56]:
def save_as_csv(users_dict, path, filename_p=""):
    
    filename = path + filename_p
    
    for user in users_dict.keys():
        
        filename += str(user) + ".csv"
        users_dict[user].to_csv(filename, index=False)
        filename = path + filename_p

In [57]:
path = "/home/marcosf/git/harena-analytics/resources/nl-experiment1/Tables/Group_DF/"
save_as_csv(groupA_clean, path)

In [58]:
path_B = "/home/marcosf/git/harena-analytics/resources/nl-experiment1/Tables/Group_EF/"
save_as_csv(groupB_clean, path_B)

# Add Physician correct answers to user track

In [61]:
correct_answers = pd.read_excel("correct_answers.xlsx", index_col=[0,1])

In [62]:
answer_case = groupA_beauti[81][groupA_beauti[81].Case == "Case-01"] 
pd.merge(answer_case, correct_answers['Points']['Case-01'], on="Answer", how="left")

,Question,Answer,Case,Points
0,EKG Hypothesis,acs,Case-01,1.0
1,EKG Confidence,48,Case-01,NaN
2,Description Hypothesis,acs,Case-01,1.0
3,Description Confidence,64,Case-01,NaN


In [63]:
correct_answers["Points"]["Case-07"]

Answer
pericarditis                         1.0
stemi                                0.0
acs                                  0.0
pericarditis / pleuritis             1.0
tendomyogeen                         0.0
stemi ipl                            0.0
costo myogeen                        0.0
inferoseptaal infarct                0.0
acs, verdenking stemi ipl            0.0
pericarditis of negroïde patiënt     1.0
musculoskeletaal                     0.0
st elevaties inferior en anterior    0.0
pericaridits                         1.0
costomyalgeen                        0.0
Name: Points, dtype: float64

In [64]:
def user_answers_points(users_track, correct_answers):
    
    users_points = dict()
    cases_list = ["Case-01", "Case-02", "Case-03", "Case-07", "Case-08", "Case-09", "Case-12", "Case-13", "Case-15", "Case-16", "Case-17", "Case-19"]
    
    for user in users_track.keys():
        users_points[user] = pd.DataFrame(columns=["Question", "Answer", "Case", "Points"])
        
        for case in cases_list:
            answer_case = users_track[user][users_track[user].Case == case]
            points_case = pd.merge(answer_case, correct_answers['Points'][case], on="Answer", how="left")   
            users_points[user] = pd.concat([users_points[user], points_case], sort=False)
        
        users_points[user].sort_values(["Question", "Case"], ascending=False)
    return users_points

In [65]:
groupA_answers_points = user_answers_points(groupA_beauti, correct_answers)

In [66]:
groupB_answers_points = user_answers_points(groupB_beauti, correct_answers)

In [67]:
groupB_answers_points[113]

,Question,Answer,Case,Points
0,EKG Hypothesis,inferolateraal negatieve ts en minimale st dep...,Case-01,0.5
1,EKG Confidence,74,Case-01,NaN
2,Description Hypothesis,inferolateraal negatieve ts en minimale st dep...,Case-01,0.5
3,Description Confidence,63,Case-01,NaN
0,EKG Hypothesis,"indien neg, vroege repolarisatie",Case-02,0.0
1,EKG Confidence,70,Case-02,NaN
2,Description Hypothesis,"nog steeds verdenking acs, opnieuw ecg maken o...",Case-02,0.0
3,Description Confidence,63,Case-02,NaN
0,EKG Hypothesis,stemi ipl,Case-03,0.0
1,EKG Confidence,71,Case-03,NaN


In [68]:
path = "/home/marcosf/git/harena-analytics/resources/nl-experiment1/Tables/Group_DF/Points"
save_as_csv(groupA_answers_points, path)

In [69]:
path = "/home/marcosf/git/harena-analytics/resources/nl-experiment1/Tables/Group_EF/Points"
save_as_csv(groupB_answers_points, path)

# Time measures

In [70]:
def users_start_time(users_track):
    
    users_time = dict()
    
    for user in users_track.keys():
        users_time[user] = pd.DataFrame(columns=["Topic", "CaseId", "Time"])
        init_timestamp = None
        for log in users_track[user]:
            if "case/Case-01/navigate" in log['topic']:
                init_timestamp = log['timestamp']
            if re.search('knot/[\w+.]+/start', log['topic']):
                var_topic = re.search('knot/[\w+.]+/start', log['topic']).group()
                case_id = log['payload']['track']['caseid']
                elapsed_time = 0
                if init_timestamp is not None:
                    elapsed_time = log['timestamp'] - init_timestamp
                    elapsed_time = elapsed_time.total_seconds()
                row = {"Topic": var_topic, "CaseId": case_id, "Time": elapsed_time}
                users_time[user] = users_time[user].append(row, ignore_index=True)
                
    return users_time

In [113]:
def time_per_cycle(users_time_track, group_kind):
    
    users_time = dict()    
    for user in users_time_track.keys():
        users_time[user] = pd.DataFrame(columns=["Cycle", "Case", "Time"])
        
        grouped = users_time_track[user].groupby("Case")
        for name, group in grouped:
            
            if group_kind == "Description First":
                
                try:
                    case_final_start = group[group["Topic"] == "knot/Final.Report/start"]["Time"]
                    case_final_start_time = case_final_start.max()
                    ekg_start = group[group["Topic"] == "knot/EKG_Cycle.EKG/start"]["Time"]
                    ekg_start_time = ekg_start.where(ekg_start < case_final_start_time).max()
                    description_start = group[group["Topic"] == "knot/Description_Cycle.Description/start"]["Time"]
                    description_start_time = description_start.where(description_start < ekg_start_time).max()
                    
                except:
                    continue

                description_cycle_time = ekg_start_time - description_start_time 
                                           
                row = {"Cycle": "Description Cycle", "Case": name, "Time": description_cycle_time}
                users_time[user] = users_time[user].append(row, ignore_index=True)
                
                ekg_cycle_time = case_final_start_time - ekg_start_time
                
                row = {"Cycle": "EKG Cycle", "Case": name, "Time": ekg_cycle_time}
                users_time[user] = users_time[user].append(row, ignore_index=True)
            
            elif group_kind == "EKG First":
                
                try:
                    case_final_start = group[group["Topic"] == "knot/Final.Report/start"]["Time"]
                    case_final_start_time = case_final_start.max()
                    description_start = group[group["Topic"] == "knot/Description_Cycle.Description/start"]["Time"]
                    description_start_time = description_start.where(description_start < case_final_start_time).max()
                    ekg_start = group[group["Topic"] == "knot/EKG_Cycle.EKG/start"]["Time"]
                    ekg_start_time = ekg_start.where(ekg_start < description_start_time).max()
                    
    
                except:
                    continue
                
                ekg_cycle_time = description_start_time - ekg_start_time
                row = {"Cycle": "EKG Cycle", "Case": name, "Time": ekg_cycle_time}
                users_time[user] = users_time[user].append(row, ignore_index=True)
                
                description_cycle_time = case_final_start_time - description_start_time  
                row = {"Cycle": "Description Cycle", "Case": name, "Time": description_cycle_time}
                users_time[user] = users_time[user].append(row, ignore_index=True)
            
            else:
                print("Insert a valid group kind")
            
    return users_time

In [72]:
def beautify_case_name(users_track, cases_name, selected_columns):
    
    for user in users_track.keys():
        df_merged = pd.merge(users_track[user],cases_name, on="CaseId", how="inner")
        df_merged = df_merged[selected_columns]
        users_track[user] = df_merged
    
    return users_track

In [73]:
cases_dict = {"Case-01":"5c0cf92e-ebe3-4f83-bb3b-11846d0f3da8",
              "Case-02":"71cb08b0-aa66-402f-8d35-027e01bd541f",
              "Case-03":"1edff734-1090-4d44-bd52-963156b6b41d",
              "Case-07":"9c5b0ab8-33c8-4bcf-bcb4-d888327b870b",
              "Case-08":"28688ea2-f8d8-4664-80f6-0926720990fb",
              "Case-09":"ec4be4fd-4e48-4e84-ba8e-36c53b415698",
              "Case-12":"280fa4a7-1e79-4097-a961-7fda24d7d9bf",
              "Case-13":"7fbf35be-79fa-4087-94cd-ccc8030ac305",
              "Case-15":"a775bf21-b15c-44d1-a120-61611bc7960c",
              "Case-16":"b90056f9-e5e2-4281-bcfe-75ab62033b2b",
              "Case-17":"070e904b-fe21-4629-83b2-4f9ea49bccce",
              "Case-19":"64634e19-ea0a-4060-9656-1252b5d6acb5"
             }
cases_name = pd.DataFrame(cases_dict.items(), columns=["Case", "CaseId"])

In [127]:
users_time_groupA = users_start_time(group_A)
users_time_groupB = users_start_time(group_B)

In [128]:
users_time_groupA_b = beautify_case_name(users_time_groupA, cases_name, selected_columns=["Topic", "Case", "Time"])
users_time_groupB_b = beautify_case_name(users_time_groupB, cases_name, selected_columns=["Topic", "Case", "Time"])

In [129]:
time_cycle_groupA = time_per_cycle(users_time_groupA_b, "Description First")

In [130]:
time_cycle_groupA[61]

,Cycle,Case,Time
0,Description Cycle,Case-01,116.289
1,EKG Cycle,Case-01,44.170
2,Description Cycle,Case-02,102.178
3,EKG Cycle,Case-02,101.369
4,Description Cycle,Case-03,81.420
5,EKG Cycle,Case-03,31.000
6,Description Cycle,Case-07,67.921
7,EKG Cycle,Case-07,34.735
8,Description Cycle,Case-08,49.271
9,EKG Cycle,Case-08,33.656


In [105]:
time_cycle_groupA[61]

,Cycle,Case,Time
0,Description Cycle,Case-01,116.289
1,EKG Cycle,Case-01,44.170
2,Description Cycle,Case-02,102.178
3,EKG Cycle,Case-02,101.369
4,Description Cycle,Case-03,81.420
5,EKG Cycle,Case-03,31.000
6,Description Cycle,Case-07,67.921
7,EKG Cycle,Case-07,34.735
8,Description Cycle,Case-08,49.271
9,EKG Cycle,Case-08,33.656


In [118]:
time_cycle_groupB = time_per_cycle(users_time_groupB_b, "EKG First")

In [132]:
users_time_groupA_b[76][users_time_groupA_b[76]["Case"] == "Case-12"].sort_values("Time")

,Topic,Case,Time
43,knot/Description_Cycle.Description/start,Case-12,536.017


In [120]:
users_time_groupB_b[108][users_time_groupB_b[108]["Case"] == "Case-13"].sort_values("Time")

,Topic,Case,Time
46,knot/EKG_Cycle.EKG/start,Case-13,912.683
47,knot/EKG_Cycle.Generate_hypothesis/start,Case-13,940.613
48,knot/Description_Cycle.Description/start,Case-13,991.375
49,knot/Description_Cycle.Generate_hypothesis/start,Case-13,991.851


In [123]:
time_cycle_groupB[86]

,Cycle,Case,Time
0,EKG Cycle,Case-01,45.655
1,Description Cycle,Case-01,126.697
2,EKG Cycle,Case-02,29.424
3,Description Cycle,Case-02,79.996
4,EKG Cycle,Case-03,26.605
5,Description Cycle,Case-03,39.663
6,EKG Cycle,Case-07,24.479
7,Description Cycle,Case-07,55.284
8,EKG Cycle,Case-08,29.708
9,Description Cycle,Case-08,43.611


In [124]:
path = "/home/marcosf/git/harena-analytics/resources/nl-experiment1/Tables/Group_DF/Time"
save_as_csv(time_cycle_groupA, path)

In [125]:
path = "/home/marcosf/git/harena-analytics/resources/nl-experiment1/Tables/Group_EF/Time"
save_as_csv(time_cycle_groupB, path)